In [ ]:
from __future__ import print_function, division
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid

from torch.utils.data import random_split
%matplotlib inline
from torchvision import transforms
from torchvision.utils import save_image
import torch.optim as optim
from PIL import Image

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


In [ ]:
from torchvision.models import VGG16_Weights

In [ ]:
# tranform_train = transforms.Compose([transforms.Resize((256,256)),transforms.RandomCrop(224,224),transforms.ColorJitter(brightness=0.5,contrast=0.5),transforms.RandomRotation(degrees=5),transforms.RandomGrayscale(0.2), transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

# #preparing the train, validation and test dataset
# # torch.manual_seed(43)
# # train_ds = CIFAR10("data/", train=True, download=True, transform=tranform_train) #40,000 original images + transforms
# # val_size = 10000 #there are 10,000 test images and since there are no transforms performed on the test, we keep the validation as 10,000
# # train_size = len(train_ds) - val_size
# # train_ds, val_ds = random_split(train_ds, [train_size, val_size]) #Extracting the 10,000 validation images from the train set
# # test_ds = CIFAR10("data/", train=False, download=True, transform=tranform_test) #10,000 images

# # #passing the train, val and test datasets to the dataloader
# # train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
# # val_dl = DataLoader(val_ds, batch_size=64, shuffle=False)
# # test_dl = DataLoader(test_ds, batch_size=64, shuffle=False)

***2.1. PREVIEWING THE TRAIN DATASET***

In [ ]:
!rm -rf ./DATASETS/
!unzip Datasets.zip 

Archive:  Datasets.zip
   creating: DATASETS/
   creating: DATASETS/7/
 extracting: DATASETS/7/Screenshot from 2022-11-29 16-32-34.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 16-33-54.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 17-08-50.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 16-26-58.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 17-23-31.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 16-37-36.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 16-31-57.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 16-57-08.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 16-57-36.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 17-20-35.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 16-52-27.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 16-52-14.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 16-37-52.png  
 extracting: DATASETS/7/Screenshot from 2022-11-29 16-41-34.png  
 extr

In [ ]:
class OCR_dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels_images = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        
        return len(self.labels_images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,str(self.labels_images.iloc[idx, 0]),
                                self.labels_images.iloc[idx, 1])
        image = Image.open(img_name).convert('RGB')
        # print("idx",idx)
        label = self.labels_images.iloc[idx, 0]
        # label = np.array([label])
        # label = label.astype('float').reshape(-1, 2)
        
        # print(type(image))
        if self.transform:
            image = self.transform(image)
        sample = {'image': image, 'label': (int(label)-1)}
        return sample

In [ ]:
# transforms.ColorJitter(brightness=0.5,contrast=0.5)

In [ ]:
tranform_train = transforms.Compose([transforms.Resize((227,227)),transforms.RandomCrop(224,224),transforms.ColorJitter(brightness=0.5,contrast=0.5,saturation=0.5,hue=0.5),transforms.RandomRotation(degrees=5),transforms.RandomGrayscale(0.2), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
val_size=24
train_ds=OCR_dataset("/content/DATASETS/label.csv","/content/DATASETS",transform=tranform_train)
train_size = len(train_ds) - val_size
train_ds, val_ds = random_split(train_ds, [train_size, val_size])
train_dl=DataLoader(train_ds, batch_size=8, shuffle=True)
val_dl=DataLoader(val_ds, batch_size=8, shuffle=True)

In [ ]:
import torchvision.models as models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = models.vgg16(weights='DEFAULT')

In [ ]:
# class OCR_dataset(Dataset):
#     """Face Landmarks dataset."""

#     def __init__(self, csv_file, root_dir, transform=None):
#         """
#         Args:
#             csv_file (string): Path to the csv file with annotations.
#             root_dir (string): Directory with all the images.
#             transform (callable, optional): Optional transform to be applied
#                 on a sample.
#         """
#         self.landmarks_frame = pd.read_csv(csv_file)
#         self.root_dir = root_dir
#         self.transform = transform

#     def __len__(self):
#         return len(self.landmarks_frame)

#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()

#         img_name = os.path.join(self.root_dir,
#                                 self.landmarks_frame.iloc[idx, 0])
#         image = io.imread(img_name)
#         landmarks = self.landmarks_frame.iloc[idx, 1:]
#         landmarks = np.array([landmarks])
#         landmarks = landmarks.astype('float').reshape(-1, 2)
        

#         if self.transform:
#             image = self.transform(image)
#         sample = {'image': image, 'landmarks': landmarks}
#         return sample

In [ ]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(25088, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 12)

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5) #dropout was included to combat overfitting
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)
        x = self.fc3(x)
        return x

***4. PREPARE THE MODEL FOR TRAINING***

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #training with either cpu or cuda

# # model = VGG16() #to compile the model
# # model = model.to(device=device) #to send the model for training on either cuda or cpu

# ## Loss and optimizer
# learning_rate = 1e-4 #I picked this because it seems to be the most used by experts
# load_model = True
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr= learning_rate) #Adam seems to be the most popular for deep learning

***5. TRAINING***

In [ ]:
model_ft.classifier[6] = nn.Linear(4096, 12, bias=True)

In [ ]:
for i,child in enumerate(model_ft.children()):
  if i==0:
    for param in child.parameters():
        param.requires_grad = False


In [ ]:
for name, param in model_ft.named_parameters():
  if not param.requires_grad:
    print(name)

features.0.weight
features.0.bias
features.2.weight
features.2.bias
features.5.weight
features.5.bias
features.7.weight
features.7.bias
features.10.weight
features.10.bias
features.12.weight
features.12.bias
features.14.weight
features.14.bias
features.17.weight
features.17.bias
features.19.weight
features.19.bias
features.21.weight
features.21.bias
features.24.weight
features.24.bias
features.26.weight
features.26.bias
features.28.weight
features.28.bias


In [ ]:
print(model_ft)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# mean=0
# std=0
# for batch_idx, sample_batched in enumerate(train_dl):
#     images=sample_batched["image"]
#     batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
#     images = images.view(batch_samples, images.size(1), -1)
#     mean += images.mean(2).sum(0)
#     std += images.std(2).sum(0)

# mean /= len(train_dl.dataset)
# std /= len(train_dl.dataset)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model_ft.to(device=device)
 #training with either cpu or cuda

# model = VGG16() #to compile the model
# model = model.to(device=device) #to send the model for training on either cuda or cpu

## Loss and optimizer
learning_rate = 1e-4 #I picked this because it seems to be the most used by experts
load_model = True
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate) #Adam seems to be the most popular for deep learning

In [ ]:
# for i_batch, sample_batched in enumerate(train_dl):
#   print(sample_batched["image"].shape)
for epoch in range(500): #I decided to train the model for 50 epochs
    loss_ep = 0
    num_correct = 0
    num_samples = 0
    for batch_idx, sample_batched in enumerate(train_dl):
        data=sample_batched["image"]
        targets=sample_batched["label"]
        data = data.to(device=device)
        targets = targets.to(device=device)
        ## Forward Pass
        optimizer.zero_grad()
        scores = model(data)
        # print("predicted data",scores.shape)
        # print("target data",targets)
        loss = criterion(scores,targets)
        loss.backward()
        optimizer.step()
        loss_ep += loss.item()
        _, predictions = scores.max(1)
        num_correct += (predictions == targets).sum()
        num_samples += predictions.size(0)
    print(f"Loss in epoch {epoch} :::: {loss_ep/len(train_dl)}")

    print(f"Got training  {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f}")
    with torch.no_grad():
        num_correct = 0
        num_samples = 0
        for batch_idx, sample_batched_val in enumerate(val_dl):
            data=sample_batched_val["image"]
            targets=sample_batched_val["label"]
            data = data.to(device=device)
            targets = targets.to(device=device)
            ## Forward Pass
            scores = model(data)
            _, predictions = scores.max(1)
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f}")

Loss in epoch 0 :::: 2.5557950122936353
Got training  28 / 296 with accuracy 9.46
Got 3 / 24 with accuracy 12.50
Loss in epoch 1 :::: 2.509066388413713
Got training  22 / 296 with accuracy 7.43
Got 3 / 24 with accuracy 12.50
Loss in epoch 2 :::: 2.4844524216007544
Got training  29 / 296 with accuracy 9.80
Got 1 / 24 with accuracy 4.17
Loss in epoch 3 :::: 2.4776858574635274
Got training  28 / 296 with accuracy 9.46
Got 2 / 24 with accuracy 8.33
Loss in epoch 4 :::: 2.518971572051177
Got training  22 / 296 with accuracy 7.43
Got 1 / 24 with accuracy 4.17
Loss in epoch 5 :::: 2.495794302708394
Got training  29 / 296 with accuracy 9.80
Got 3 / 24 with accuracy 12.50
Loss in epoch 6 :::: 2.497205727809184
Got training  33 / 296 with accuracy 11.15
Got 4 / 24 with accuracy 16.67
Loss in epoch 7 :::: 2.5043677768191777
Got training  24 / 296 with accuracy 8.11
Got 2 / 24 with accuracy 8.33
Loss in epoch 8 :::: 2.521695710517265
Got training  25 / 296 with accuracy 8.45
Got 3 / 24 with accura

***6.SAVING THE MODEL AND USING IT FOR THE TEST SET***

In [ ]:
model = VGG16()
model.load_state_dict(torch.load("/content/vgg16_cifar.pt")) #loads the trained model
model.eval()

FileNotFoundError: ignored

***7. TESTING***

In [ ]:
num_correct = 0
num_samples = 0
for batch_idx, (data,targets) in enumerate(test_dl):
    data = data.to(device="cpu")
    targets = targets.to(device="cpu")
    ## Forward Pass
    scores = model(data)
    _, predictions = scores.max(1)
    num_correct += (predictions == targets).sum()
    num_samples += predictions.size(0)
print(
    f"Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f}"
)

In [ ]:
!wget https://www.openslr.org/resources/43/ne_np_female.zip


--2022-12-21 11:22:38--  https://www.openslr.org/resources/43/ne_np_female.zip
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/43/ne_np_female.zip [following]
--2022-12-21 11:22:39--  http://us.openslr.org/resources/43/ne_np_female.zip
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 800375645 (763M) [application/zip]
Saving to: ‘ne_np_female.zip’

ne_np_female.zip    100%[===================>] 763.30M  20.4MB/s    in 38s     

2022-12-21 11:23:17 (20.1 MB/s) - ‘ne_np_female.zip’ saved [800375645/800375645]



In [ ]:
!unzip /content/ne_np_female.zip -d ./data

Archive:  /content/ne_np_female.zip
   creating: ./data/ne_np_female/
  inflating: ./data/ne_np_female/line_index.tsv  
   creating: ./data/ne_np_female/wavs/
  inflating: ./data/ne_np_female/wavs/nep_0258_0119737288.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_0461984530.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_0576399736.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_0707220145.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_0838627393.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_1082186644.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_1139532706.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_2044831682.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_2658340102.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_2670176669.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_2764563824.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_2784581247.wav  
  inflating: ./data/ne_np_female/wavs/nep_0258_2857262530.wav  
  inflati